In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from DeepScence.api import DeepScence
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from dca.api import dca
import os
os.chdir(b'/Users/lele/Library/Mobile Documents/com~apple~CloudDocs/Research/Aging')

我困得要死


#### collect scores

In [2]:
hayflick = sc.read_h5ad("./data/in_vitro/scored_h5ad/subsets/hayflick_dca.h5ad")
hca = sc.read_h5ad("./data/in_vitro/scored_h5ad/subsets/hca_dca.h5ad")
huvec = sc.read_h5ad("./data/in_vitro/scored_h5ad/subsets/huvec_dca.h5ad")
notch = sc.read_h5ad("./data/in_vitro/scored_h5ad/subsets/notch_dca.h5ad")
oskm = sc.read_h5ad("./data/in_vitro/scored_h5ad/subsets/oskm_dca.h5ad")

In [3]:
adata_dict = {
    "hayflick": hayflick,
    "hca": hca,
    "notch": notch,
    "oskm": oskm,
    "huvec": huvec
}
for name, adata in adata_dict.items():
    
    other_scores = pd.read_csv(f"./data/in_vitro/scores/{name}_other_scores.csv", index_col=0)
    cid_scores = pd.read_csv(f"./data/in_vitro/scores/{name}_SenCID.csv", index_col=0)
    other_scores = other_scores.loc[adata.obs_names]
    cid_scores = cid_scores.loc[adata.obs_names]
    
    gene_expression_df = pd.DataFrame({
        "CDKN1A_positive": (adata[:, "CDKN1A"].X > 0).flatten().astype(int),
        "CDKN2A_positive": (adata[:, "CDKN2A"].X > 0).flatten().astype(int)
    }, index=adata.obs.index)
    
    combined_scores = pd.concat([other_scores, cid_scores, gene_expression_df], axis=1)
    adata.obs = adata.obs.drop(columns=combined_scores.columns, errors='ignore')
    adata.obs = pd.concat([adata.obs, combined_scores], axis=1)
    adata_dict[name] = adata

#### 1. Add DeepScence scores

In [4]:
args = {
        "binarize": True,
        "verbose": False,
        "denoise": False,
        "lambda_ortho": 1,
        "random_state": 0
    }
for name, adata in adata_dict.items():
    print(f"processing {name}")

    # keep high quality only
    adata = adata[adata.obs["clean_label"]!="bad"]
    adata = DeepScence(adata, **args)
    adata_dict[name] = adata

processing hayflick


[2024-10-01 11:30] Input is preprocessed, preprocessed 14317 genes and 2869 cells.
[2024-10-01 11:30] Using 34 genes in the gene set for scoring
[2024-10-01 11:30] Lambda provided, capturing scores in 2 neurons.
[2024-10-01 11:30] Training on 2583 cells, validate on 286 cells.
[2024-10-01 11:30] Binarizing with permutation...
100%|███████████████████████████████████████████| 50/50 [00:18<00:00,  2.67it/s]
/Users/lele/Downloads/anaconda3/envs/sene/lib/python3.8/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
[2024-10-01 11:31] Input is preproc

processing hca


[2024-10-01 11:31] Using 35 genes in the gene set for scoring
[2024-10-01 11:31] Lambda provided, capturing scores in 2 neurons.
[2024-10-01 11:31] Training on 711 cells, validate on 79 cells.
[2024-10-01 11:31] Binarizing with permutation...
100%|███████████████████████████████████████████| 50/50 [00:04<00:00, 11.91it/s]
[2024-10-01 11:31] Input is preprocessed, preprocessed 15344 genes and 392 cells.
[2024-10-01 11:31] Using 35 genes in the gene set for scoring
[2024-10-01 11:31] Lambda provided, capturing scores in 2 neurons.
[2024-10-01 11:31] Training on 353 cells, validate on 39 cells.


processing notch


[2024-10-01 11:31] Binarizing with permutation...
100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 15.06it/s]
[2024-10-01 11:31] Input is preprocessed, preprocessed 15509 genes and 198 cells.
[2024-10-01 11:31] Using 34 genes in the gene set for scoring
[2024-10-01 11:31] Lambda provided, capturing scores in 2 neurons.
[2024-10-01 11:31] Training on 179 cells, validate on 19 cells.


processing oskm


[2024-10-01 11:31] Binarizing with permutation...
100%|███████████████████████████████████████████| 50/50 [00:02<00:00, 21.48it/s]


processing huvec


[2024-10-01 11:31] Input is preprocessed, preprocessed 11625 genes and 13518 cells.
[2024-10-01 11:31] Using 34 genes in the gene set for scoring
[2024-10-01 11:31] Lambda provided, capturing scores in 2 neurons.
[2024-10-01 11:31] Training on 12167 cells, validate on 1351 cells.
[2024-10-01 11:32] Binarizing with permutation...
100%|███████████████████████████████████████████| 50/50 [02:25<00:00,  2.91s/it]


In [7]:
adata_dict["oskm"].obs.columns

Index(['condition', 'SnC', 'p16', 'p21', 'p53', 'clean_label', 'dca_split',
       'n_counts', 'size_factors', 'ssGSEA_CoreScence', 'ssGSEA_senmayo',
       'ssGSEA_cellAge', 'ssGSEA_geneAge', 'ssGSEA_csgene', 'ssGSEA_cellsig',
       'ssGSEA_quest', 'ssGSEA_sasp', 'ssGSEA_trans', 'AUCell_CoreScence',
       'AUCell_senmayo', 'AUCell_cellAge', 'AUCell_geneAge', 'AUCell_csgene',
       'AUCell_cellsig', 'AUCell_quest', 'AUCell_sasp', 'AUCell_trans',
       'CDKN1A', 'CDKN2A', 'senmayo_binary', 'SID_score', 'CDKN1A_positive',
       'CDKN2A_positive', 'ds', 'p', 'cluster', 'binary'],
      dtype='object')

In [8]:
scores = ['ssGSEA_CoreScence', 'ssGSEA_senmayo',
       'ssGSEA_cellAge', 'ssGSEA_geneAge', 'ssGSEA_csgene', 'ssGSEA_cellsig',
       'ssGSEA_quest', 'ssGSEA_sasp', 'ssGSEA_trans', 'AUCell_CoreScence',
       'AUCell_senmayo', 'AUCell_cellAge', 'AUCell_geneAge', 'AUCell_csgene',
       'AUCell_cellsig', 'AUCell_quest', 'AUCell_sasp', 'AUCell_trans',
       'CDKN1A', 'CDKN2A', 'SID_score', "ds"]
results = []
for name, adata in adata_dict.items():
    adata.obs.to_csv(f"./data/in_vitro/benchmark/{name}_scored_meta.csv", index=True)
    aucs = []
    for col in scores:
        if col in adata.obs.columns:
            adata.obs[col] = adata.obs[col].fillna(0)
            auroc = roc_auc_score(adata.obs["SnC"].values, adata.obs[col].values)
        else:
            auroc = np.nan
        aucs.append(auroc)
    results.append(aucs)
results = pd.DataFrame(results)
results.index = list(adata_dict.keys())
results.columns = scores
results.to_csv("./data/in_vitro/benchmark/benchmark_results.csv", index=True)
results

,ssGSEA_CoreScence,ssGSEA_senmayo,ssGSEA_cellAge,ssGSEA_geneAge,ssGSEA_csgene,ssGSEA_cellsig,ssGSEA_quest,ssGSEA_sasp,ssGSEA_trans,AUCell_CoreScence,...,AUCell_geneAge,AUCell_csgene,AUCell_cellsig,AUCell_quest,AUCell_sasp,AUCell_trans,CDKN1A,CDKN2A,SID_score,ds
hayflick,0.915726,0.914359,0.750198,0.651831,0.557229,0.983637,0.963584,0.887574,0.978452,0.986705,...,0.505259,0.412433,0.998707,0.993317,0.948945,0.995033,0.963411,0.520346,0.981345,0.999750
hca,0.690203,0.338459,0.503355,0.487178,0.474007,0.879282,0.624554,0.353591,0.799878,0.900202,...,0.507198,0.563019,0.991665,0.819551,0.509579,0.885002,0.727403,0.469117,0.969627,0.999378
notch,0.940655,0.948089,0.884283,0.688692,0.594678,0.823060,0.876614,0.971879,0.850241,0.818312,...,0.399270,0.345807,0.775792,0.776431,0.868710,0.731199,0.749915,0.750020,0.964888,0.985392
oskm,0.775953,0.444174,0.505826,0.450847,0.454555,0.786653,0.740148,0.453496,0.760805,0.844174,...,0.252013,0.243750,0.969809,0.919492,0.442479,0.825583,0.691102,0.499576,0.935381,0.970551
huvec,0.764739,0.797511,0.607325,0.518427,0.485457,0.837814,0.805034,0.703443,0.662339,0.853887,...,0.454555,0.333911,0.914253,0.889547,0.827271,0.669925,0.652601,0.769341,0.911363,0.932809


In [11]:
binary = ["CDKN1A_positive", "CDKN2A_positive", "senmayo_binary", "SID_score", "binary"]
results = []
f1_results = []
for name, adata in adata_dict.items():
    accs = []
    f1s = []
    for col in binary:
        if col in adata.obs.columns:
            adata.obs[col] = adata.obs[col].fillna(0)
            if col != "SID_score":
                predictions = (adata.obs[col].values).astype(int)
            else:
                predictions = (adata.obs[col].values > 0.5).astype(int)
            accuracy = accuracy_score(adata.obs["SnC"].values, predictions)
            f1 = f1_score(adata.obs["SnC"].values, predictions)
        else:
            accuracy = np.nan
            f1 = np.nan
        accs.append(accuracy)
        f1s.append(f1)
    results.append(accs)
    f1_results.append(f1s)
results = pd.DataFrame(results)
results.index = list(adata_dict.keys())
results.columns = binary

f1_df = pd.DataFrame(f1_results)
f1_df.index = list(adata_dict.keys())
f1_df.columns = binary

results.to_csv("./data/in_vitro/benchmark/benchmark_acc_results.csv", index=True)
f1_df.to_csv("./data/in_vitro/benchmark/benchmark_f1_results.csv", index=True)
results

,CDKN1A_positive,CDKN2A_positive,senmayo_binary,SID_score,binary
hayflick,0.329383,0.329383,0.821889,0.927152,0.993726
hca,0.500000,0.500000,0.486076,0.779747,0.994937
notch,0.477041,0.477041,0.640306,0.869898,0.920918
oskm,0.404040,0.404040,0.545455,0.893939,0.919192
huvec,0.389555,0.389555,0.673620,0.824234,0.867214


### Try

In [ ]:
# adata = notch.copy()
adata = sc.read_h5ad("./data/in_vitro/sc_notch/notch.h5ad")
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

In [ ]:
adata.var

In [ ]:
plt.hist(adata.var["dispersions"])
plt.show()
adata.var["highly_variable"].value_counts()